# Model testing

In [2]:
import torch
import math

True
True


In [1]:
import torch
from models import ConvNet1D
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle
import pandas as pd
import nltk

## Test performance

In [2]:
# Load test dataset
test_dataset = torch.load('data/test_w2v.pt')
test_loader = DataLoader(test_dataset)

In [3]:
# Model parameters
input_channels = 50
depth = 0
kernel_size = 5
max_essay_length = 1266
skip_connections = False
batch_norm = False

In [4]:
# Load model weights
model = ConvNet1D(input_channels, depth, kernel_size, max_essay_length, skip_connections, batch_norm)
weights = torch.load('models/ConvNet1D_depth0_seed1.pth', map_location=torch.device('cpu'))
model.load_state_dict(weights)

<All keys matched successfully>

In [5]:
# Test performance (MAE)
test_sae = 0

with torch.no_grad():
    for essay, essay_set, grade in test_loader:
        grade_pred = model(essay, essay_set)
        test_sae += torch.abs(grade_pred-grade).item()

test_mae = test_sae/len(test_dataset)
print(f'Test MAE: {test_mae:.4f}')

Test MAE: 0.1045


## Example of essay grading

In [6]:
# Load W2V dictionary
with open('data/dict_w2v.pickle', 'rb') as f:
    dict_w2v = pickle.load(f)

In [7]:
# Load data
data = pd.read_excel('data/training_set_rel3.xlsx')

In [8]:
# Essay to embedding function
def essay_to_emb(essay):
    sentences = nltk.sent_tokenize(essay)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    emb_essay = np.zeros([50,1266])
    counter = 0

    for sentence in tokenized_sentences:
        for word in sentence[:-1]:
            try:
                emb_essay[:,counter] = dict_w2v[word]
                counter += 1
            except:
                if word != ',':
                    emb_essay[:,counter] = np.zeros(50)
                    counter += 1
                else:
                    continue
                   
        emb_essay[:,counter] = dict_w2v['</s>']
        counter += 1
    
    emb_essay = torch.tensor(emb_essay)
    
    return emb_essay.view(1, emb_essay.shape[0], emb_essay.shape[1])

In [9]:
# Display subject, essay, true grade and predicted grade
idx = 0
essay = essay_to_emb(data.essay[idx]).float()
essay_set = torch.tensor(data.essay_set[idx]).view(-1)
grade = torch.tensor((data.domain1_score[idx]-2)/10).view(-1).float()
print(f'Subject:\nMore and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.\n')
print(f'Essay:\n{data.essay[idx]}\n')
print(f'True grade: {grade.item()*100:.1f}%\n')
print(f'Predicted grade: {model(essay, essay_set).detach().item()*100:.1f}%')

Subject:
More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.

Essay:
Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenag